In [53]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [57]:
train_data = pd.read_csv('./refined_data/train_data.csv')
test_data = pd.read_csv('./refined_data/sample_submission.csv')
booking_data = pd.read_csv('./refined_data/bookings_data.csv')
booking_data_2 = pd.read_csv('./refined_data/bookings.csv')
hotel_data = pd.read_csv('./refined_data/hotels_data.csv')
customer_data = pd.read_csv('./refined_data/customer_data.csv')
payments_data = pd.read_csv('./refined_data/payments_data.csv')

# merge booking data and booking data 2 according to the booking id
booking_data_combined = pd.merge(booking_data, booking_data_2, on='booking_id')
booking_data_combined.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,customer_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date
count,112650,112650.000000,112650,112650,112650,112650.000000,112650.000000,112650,112650,112650,112635,110196
unique,98666,NaN,71496,17141,93318,NaN,NaN,98666,7,98112,90174,95664
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,b24856965c36a24e339b605899a4788c,19b47992c3666cc44a7e94c06560211a,2017-07-21 18:25:23,NaN,NaN,19d62d49bfb5e1f83123e9fc3d1daec3,completed,2007-07-19 18:02:25,2008-02-27 03:03:27,2007-08-03 17:46:02
freq,21,NaN,68,222,21,NaN,NaN,21,110197,21,23,21
mean,NaN,1.197834,NaN,NaN,NaN,120.653739,19.990320,NaN,NaN,NaN,NaN,NaN
std,NaN,0.705124,NaN,NaN,NaN,183.633928,15.806405,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.000000,NaN,NaN,NaN,39.900000,13.080000,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.000000,NaN,NaN,NaN,74.990000,16.260000,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.000000,NaN,NaN,NaN,134.900000,21.150000,NaN,NaN,NaN,NaN,NaN


In [55]:
# create expiry_columns for year, month, day, hour, minute, second
booking_data_combined['expiry_year'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).year
booking_data_combined['expiry_month'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).month
booking_data_combined['expiry_day'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).day
booking_data_combined['expiry_hour'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).hour
booking_data_combined['expiry_minute'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).minute
booking_data_combined['expiry_second'] = pd.DatetimeIndex(booking_data_combined['booking_expiry_date']).second

# remove booking_expiry_date column
booking_data_combined = booking_data_combined.drop(['booking_expiry_date'], axis=1)

# create create_columns for year, month, day, hour, minute, second
booking_data_combined['create_year'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).year
booking_data_combined['create_month'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).month
booking_data_combined['create_day'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).day
booking_data_combined['create_hour'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).hour
booking_data_combined['create_minute'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).minute
booking_data_combined['create_second'] = pd.DatetimeIndex(booking_data_combined['booking_create_timestamp']).second

# remove booking_create_timestamp column
booking_data_combined = booking_data_combined.drop(['booking_create_timestamp'], axis=1)

# create checkin_columns for year, month, day, hour, minute, second
booking_data_combined['checkin_year'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).year
booking_data_combined['checkin_month'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).month
booking_data_combined['checkin_day'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).day
booking_data_combined['checkin_hour'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).hour
booking_data_combined['checkin_minute'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).minute
booking_data_combined['checkin_second'] = pd.DatetimeIndex(booking_data_combined['booking_checkin_customer_date']).second

# remove booking_checkin_customer_date column
booking_data_combined = booking_data_combined.drop(['booking_checkin_customer_date'], axis=1)

# create approved_columns for year, month, day, hour, minute, second

booking_data_combined['approved_year'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).year
booking_data_combined['approved_month'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).month
booking_data_combined['approved_day'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).day
booking_data_combined['approved_hour'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).hour
booking_data_combined['approved_minute'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).minute
booking_data_combined['approved_second'] = pd.DatetimeIndex(booking_data_combined['booking_approved_at']).second

# remove booking_approved_at column
booking_data_combined = booking_data_combined.drop(['booking_approved_at'], axis=1)

# change booking statius to 1 if completed, 2, if invoiced, 3 if cancelled

print(booking_data_combined['booking_status'].unique())
count_each_status = booking_data_combined['booking_status'].value_counts()
print(count_each_status)

['completed' 'processed' 'canceled' 'invoiced' 'processing' 'approved'
 'unavailable']
completed      110197
processed        1185
canceled          542
invoiced          359
processing        357
unavailable         7
approved            3
Name: booking_status, dtype: int64
